# Feature Engineering

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf

In [4]:
STOCKS = {
    "US": ["AAPL", "MSFT", "AMZN", "NVDA", "JPM"],
    "Canada": ["RY.TO", "TD.TO", "ENB.TO", "CM.TO", "NA.TO"],
    "India": ["RELIANCE.NS", "TCS.NS", "INFY.NS","ADANIPOWER.NS", "HDFCBANK.NS"],
}
# Since we are building a multi-stock model, we will gather stock data from all 3 countries markets to help the model generalize better.

In [5]:
def prepare_stock_df(ticker, start="2010-01-01", end="2025-12-31"):
    df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)
    if df is None or df.empty:
        return None

    # If yfinance ever returns MultiIndex columns, flatten them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df = df.reset_index()  # adds Date column
    close = df["Close"].squeeze() # makes sure to make a series instead of dataframe

    # Returns
    df["ret"] = close.pct_change()
    df["ret_1"] = df["ret"].shift(1)
    df["ret_5"] = df["ret"].shift(5)
    df["ret_20"] = df["ret"].shift(20)

    # Volatility
    df["vol20"] = df["ret"].rolling(20).std()

    # RSI
    delta = close.diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = (-delta.clip(upper=0)).rolling(14).mean()
    rs = gain / loss
    df["RSI14"] = 100 - (100 / (1 + rs))

    # MACD
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    df["MACD"] = ema12 - ema26

    # Bollinger width (normalized)
    m20 = close.rolling(20).mean()
    s20 = close.rolling(20).std()
    df["bb_width"] = (4 * s20) / m20   # (BB_up - BB_low) / mid

    # ATR %
    high, low = df["High"].squeeze(), df["Low"].squeeze()
    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

    df["ATR14"] = tr.rolling(14).mean()
    df["atr_pct"] = df["ATR14"] / close

    # Target: next-day return
    df["target"] = df["ret"].shift(-1)

    df["ticker"] = ticker

    keep = ["Date", "ticker", "Close", "High", "Low", "Open", "Volume",
            "ret", "ret_1", "ret_5", "ret_20", "vol20", "RSI14", "MACD",
            "bb_width", "ATR14", "atr_pct", "target"]

    df = df[keep].dropna()

    return df

In [6]:
all_data = []

for market, tickers in STOCKS.items():
    for t in tickers:
        print(f"Fetching {t}")
        df_t = prepare_stock_df(t)
        if df_t is not None and not df_t.empty:
            all_data.append(df_t)

full_df = pd.concat(all_data, ignore_index=True)

Fetching AAPL
Fetching MSFT
Fetching AMZN
Fetching NVDA
Fetching JPM
Fetching RY.TO
Fetching TD.TO
Fetching ENB.TO
Fetching CM.TO
Fetching NA.TO
Fetching RELIANCE.NS
Fetching TCS.NS
Fetching INFY.NS
Fetching ADANIPOWER.NS
Fetching HDFCBANK.NS


In [7]:
full_df[full_df['ticker']=='ADANIPOWER.NS']

Price,Date,ticker,Close,High,Low,Open,Volume,ret,ret_1,ret_5,ret_20,vol20,RSI14,MACD,bb_width,ATR14,atr_pct,target
51744,2010-02-03,ADANIPOWER.NS,20.889999,21.059999,20.320000,20.540001,12636495,0.026031,-0.004888,-0.024951,0.061111,0.015304,60.240985,0.071562,0.082942,0.659286,0.031560,0.003830
51745,2010-02-04,ADANIPOWER.NS,20.969999,21.280001,20.770000,21.000000,15340595,0.003830,0.026031,-0.002509,0.022846,0.014360,63.967598,0.116528,0.072293,0.660715,0.031508,-0.008107
51746,2010-02-05,ADANIPOWER.NS,20.799999,20.950001,20.150000,20.400000,9830550,-0.008107,0.003830,0.006036,-0.023267,0.013505,59.386973,0.136868,0.069742,0.699286,0.033620,0.019712
51747,2010-02-08,ADANIPOWER.NS,21.209999,21.770000,20.940001,20.940001,11656500,0.019712,-0.008107,0.023000,-0.011910,0.013927,66.894203,0.183950,0.076796,0.744286,0.035091,0.019331
51748,2010-02-09,ADANIPOWER.NS,21.620001,22.000000,21.049999,21.200001,15257645,0.019331,0.019712,-0.004888,0.020251,0.013862,72.699407,0.251449,0.087349,0.785000,0.036309,-0.012488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55667,2025-12-23,ADANIPOWER.NS,144.300003,144.600006,142.320007,143.800003,7437606,0.007330,0.010511,0.006580,0.009626,0.013632,48.604753,-1.884431,0.074627,3.406430,0.023607,-0.007900
55668,2025-12-24,ADANIPOWER.NS,143.160004,144.699997,142.669998,144.699997,7905799,-0.007900,0.007330,-0.016032,0.012122,0.013367,50.938471,-1.769602,0.068509,3.227858,0.022547,0.000000
55669,2025-12-25,ADANIPOWER.NS,143.160004,143.160004,143.160004,143.160004,0,0.000000,-0.007900,-0.011888,-0.003297,0.013368,48.581897,-1.659471,0.062323,3.097144,0.021634,-0.006776
55670,2025-12-26,ADANIPOWER.NS,142.190002,144.369995,141.899994,143.149994,7682604,-0.006776,0.000000,0.003255,-0.004321,0.013405,58.706487,-1.631653,0.057039,2.777144,0.019531,-0.012307


In [8]:
full_df.columns

Index(['Date', 'ticker', 'Close', 'High', 'Low', 'Open', 'Volume', 'ret',
       'ret_1', 'ret_5', 'ret_20', 'vol20', 'RSI14', 'MACD', 'bb_width',
       'ATR14', 'atr_pct', 'target'],
      dtype='object', name='Price')

In [9]:
FEATURES = [
    'ret','ret_1', 'ret_5', 'ret_20', 'vol20', 'RSI14', 
    'MACD', 'bb_width', 'ATR14', 'atr_pct'
]
# We dont use the OHLC columns as they are non-stationary, redundant and harder to learn for LSTMs.

split_date = "2022-01-01"

train_df = full_df[full_df['Date'] < split_date] # Training data from 2010-2022
test_df = full_df[full_df['Date'] >= split_date] # Testing data from 2022-2025

X_train = train_df[FEATURES]
y_train = train_df["target"]

X_test = test_df[FEATURES]
y_test = test_df["target"]

# TRAINING THE MODEL(RANDOM FOREST REGRESSOR)

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(
    n_estimators=400,
    max_depth=8,
    min_samples_leaf=50,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

,n_estimators,400
,criterion,'squared_error'
,max_depth,8
,min_samples_split,2
,min_samples_leaf,50
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [11]:
preds = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, preds))

MAE: 0.011897248553432356


In [12]:
# The above MAE represents that the daily return predictions have an error margin of around +/- 1.18%

In [13]:
naive_preds = X_test["ret_1"].values
print("Naive MAE:", mean_absolute_error(y_test, naive_preds))

Naive MAE: 0.0171016937709074


In [14]:
direction_acc = (np.sign(preds) == np.sign(y_test.values)).mean()
print("Directional accuracy:", direction_acc)

Directional accuracy: 0.5276254180602007


In [ ]:
import numpy as np
import pandas as pd

def max_drawdown(equity_curve: pd.Series) -> float:
    # Max drawdown in % terms (negative number).
    peak = equity_curve.cummax()
    dd = equity_curve / peak - 1.0
    return float(dd.min())

def sharpe_ratio(daily_returns: pd.Series, rf_daily: float = 0.0) -> float:
    
    # Annualized Sharpe ratio based on daily returns.
    
    r = daily_returns.dropna() - rf_daily
    if r.std() == 0 or len(r) < 2:
        return float("nan")
    return float((r.mean() / r.std()) * np.sqrt(252))

def run_backtest(
    test_df: pd.DataFrame,
    preds: np.ndarray,
    cost_bps: float = 5.0,
    mode: str = "long_cash",
    signal_threshold: float = 0.0005,
) -> pd.DataFrame:
    """
    Backtest a simple daily strategy with a confidence threshold.

    Parameters
    ----------
    test_df : DataFrame
        Must contain ['Date','ticker','target'] where target = next-day return.
    preds : np.ndarray
        Predicted next-day returns aligned with test_df rows.
    cost_bps : float
        Transaction cost per position change in basis points (bps).
    mode : str
        'long_cash' -> position in {0, +1}
        'long_short'-> position in {-1, 0, +1} with symmetric thresholds
    signal_threshold : float
        Confidence threshold on predicted return.
        - long_cash: go long if pred > +threshold else cash
        - long_short: long if pred > +threshold, short if pred < -threshold, else flat

    Returns
    -------
    DataFrame with daily strategy returns, equity curves, plus exposure/turnover stats.
    """

    bt = test_df[["Date", "ticker", "target"]].copy()
    bt["pred"] = np.asarray(preds)

    # Build positions using the threshold (DO NOT override the arg)
    if mode == "long_cash":
        # Long if confident positive prediction, else cash
        bt["pos"] = (bt["pred"] > signal_threshold).astype(int)  # 1 or 0

    elif mode == "long_short":
        # Long if > +th, short if < -th, else flat (0)
        bt["pos"] = np.where(
            bt["pred"] > signal_threshold, 1,
            np.where(bt["pred"] < -signal_threshold, -1, 0)
        )

    else:
        raise ValueError("mode must be 'long_cash' or 'long_short'")

    # Sort so turnover is computed correctly per ticker over time
    bt = bt.sort_values(["ticker", "Date"]).reset_index(drop=True)

    # Turnover = position changes (per ticker)
    bt["pos_prev"] = bt.groupby("ticker")["pos"].shift(1)
    bt["turnover"] = (bt["pos"] - bt["pos_prev"]).abs().fillna(0)

    # Transaction costs
    cost = cost_bps / 10000.0  # bps -> decimal
    bt["cost"] = bt["turnover"] * cost

    # Strategy returns (per ticker)
    bt["strat_ret"] = bt["pos"] * bt["target"] - bt["cost"]
    bt["bh_ret"] = bt["target"]

    # Portfolio aggregation (equal-weight across tickers each day) 
    daily = (
        bt.groupby("Date", as_index=False)
          .agg(
              strat_ret=("strat_ret", "mean"),
              bh_ret=("bh_ret", "mean"),
              # Diagnostics:
              exposure=("pos", lambda x: float((x != 0).mean())),   # % tickers invested that day
              avg_abs_pos=("pos", lambda x: float(np.mean(np.abs(x)))),  # similar to exposure
              avg_turnover=("turnover", "mean"),
          )
          .sort_values("Date")
    )

    daily["strat_equity"] = (1 + daily["strat_ret"]).cumprod()
    daily["bh_equity"] = (1 + daily["bh_ret"]).cumprod()

    return daily

def summarize_backtest(daily: pd.DataFrame, label: str = "") -> dict:
    # Compute key metrics from daily backtest results.
    strat_total = float(daily["strat_equity"].iloc[-1] - 1)
    bh_total = float(daily["bh_equity"].iloc[-1] - 1)

    strat_sharpe = sharpe_ratio(daily["strat_ret"])
    bh_sharpe = sharpe_ratio(daily["bh_ret"])

    strat_mdd = max_drawdown(daily["strat_equity"])
    bh_mdd = max_drawdown(daily["bh_equity"])

    # Added diagnostics
    exposure_avg = float(daily["exposure"].mean()) if "exposure" in daily.columns else float("nan")
    turnover_avg = float(daily["avg_turnover"].mean()) if "avg_turnover" in daily.columns else float("nan")

    return {
        "Label": label,
        "Strategy Total Return": strat_total,
        "Buy&Hold Total Return": bh_total,
        "Strategy Sharpe": strat_sharpe,
        "Buy&Hold Sharpe": bh_sharpe,
        "Strategy Max Drawdown": strat_mdd,
        "Buy&Hold Max Drawdown": bh_mdd,
        "Avg Exposure": exposure_avg,       # how often we are actually in trades
        "Avg Turnover": turnover_avg,       # how much we trade (cost driver)
        "Days": int(len(daily)),
    }

In [16]:
'''
thresholds = [0.0, 0.0005, 0.001, 0.002, 0.003, 0.005, 0.01]
rows = []

for th in thresholds:
    daily = run_backtest(test_df_aligned, preds, cost_bps=5.0, mode="long_cash", signal_threshold=th)
    rows.append(summarize_backtest(daily, label=f"th={th}"))

pd.DataFrame(rows).sort_values("Strategy Sharpe", ascending=False)
'''

'\nthresholds = [0.0, 0.0005, 0.001, 0.002, 0.003, 0.005, 0.01]\nrows = []\n\nfor th in thresholds:\n    daily = run_backtest(test_df_aligned, preds, cost_bps=5.0, mode="long_cash", signal_threshold=th)\n    rows.append(summarize_backtest(daily, label=f"th={th}"))\n\npd.DataFrame(rows).sort_values("Strategy Sharpe", ascending=False)\n'

In [ ]:
test_df_aligned = test_df.loc[X_test.index].copy()

daily_lc = run_backtest(
    test_df=test_df_aligned,
    preds=preds,
    cost_bps=5.0,             # 5 bps per position change
    mode="long_cash",
    signal_threshold=0.0005
)

daily_ls = run_backtest(
    test_df=test_df_aligned,
    preds=preds,
    cost_bps=5.0,
    mode="long_short",
    signal_threshold=0.0005
)

summary = pd.DataFrame([
    summarize_backtest(daily_lc, "Long/Cash (5 bps)"),
    summarize_backtest(daily_ls, "Long/Short (5 bps)"),
])

summary

,Label,Strategy Total Return,Buy&Hold Total Return,Strategy Sharpe,Buy&Hold Sharpe,Strategy Max Drawdown,Buy&Hold Max Drawdown,Avg Exposure,Avg Turnover,Days
0,Long/Cash (5 bps),0.997257,1.124629,1.436383,1.372811,-0.146205,-0.169536,0.908398,0.143951,1036
1,Long/Short (5 bps),0.907919,1.124629,1.359989,1.372811,-0.153893,-0.169536,0.924131,0.166345,1036


In [18]:
import plotly.graph_objects as go

def plot_equity_curve(daily: pd.DataFrame, title: str):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=daily["Date"], y=daily["strat_equity"], name="Strategy"))
    fig.add_trace(go.Scatter(x=daily["Date"], y=daily["bh_equity"], name="Buy & Hold"))
    fig.update_layout(title=title, height=420, hovermode="x unified")
    fig.show()

plot_equity_curve(daily_lc, "Equity Curve — Long/Cash vs Buy & Hold")
plot_equity_curve(daily_ls, "Equity Curve — Long/Short vs Buy & Hold")

In [19]:
# Through the above graphs its pretty clear that using a threshold filter helped our strategy significantly reduce drawdowns and improve overall returns.
# The model improves risk-adjusted performance and drawdown protection but underperforms in late-stage bull markets.
# The model was performing better until April 8th 2025, around the time when Trump announced reciprocal tariffs.

In [20]:
# Tried using GARCH volatility model as a feature to see if it improves model performance
# But it didnt help much and instead made our model worse, so we will try using LSTM model to see if it helps.
# The reason GARCH didnt help could be because we are trying to predict next-day returns but GARCH predicts volatility trends, 
# and volatility != predictions, therefore GARCH adds noise which reduced our model Sharpe ratio.

In [22]:
full_df.head()

Price,Date,ticker,Close,High,Low,Open,Volume,ret,ret_1,ret_5,ret_20,vol20,RSI14,MACD,bb_width,ATR14,atr_pct,target
0,2010-02-03,AAPL,5.975114,6.004206,5.830857,5.853351,615328000,0.017206,0.005803,0.009420,0.001729,0.023284,41.111473,-0.125231,0.129604,0.221420,0.037057,-0.036038
1,2010-02-04,AAPL,5.759781,5.949324,5.745385,5.900138,757652000,-0.036038,0.017206,-0.041322,-0.015906,0.024267,37.621167,-0.138023,0.139852,0.234337,0.040685,0.017755
2,2010-02-05,AAPL,5.862049,5.878244,5.723790,5.777174,850306800,0.017755,-0.036038,-0.036279,-0.001849,0.024767,42.533131,-0.138315,0.142663,0.233095,0.039763,-0.006856
3,2010-02-08,AAPL,5.821861,5.934628,5.818262,5.868947,478270800,-0.006856,0.017755,0.013902,0.006649,0.024661,33.221056,-0.140173,0.143807,0.221569,0.038058,0.010663
4,2010-02-09,AAPL,5.883942,5.923230,5.840755,5.890840,632886800,0.010663,-0.006856,0.005803,-0.008822,0.024849,37.283141,-0.135078,0.142919,0.215850,0.036685,-0.005454


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib, json
from pathlib import Path


# CONFIG
SEQ_LEN = 60
TRAIN_END = "2022-01-01"   # train < this date, test >= this date
FEATURES = [
    "ret", "ret_1", "ret_5", "ret_20",
    "vol20", "RSI14", "MACD", "bb_width", "ATR14", "atr_pct"
]
TARGET_COL = "target"      # next-day return

# Basic cleaning + sort
df = full_df.copy()
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["ticker","Date"]).reset_index(drop=True)

# Drop rows with missing features/target
df = df.dropna(subset=FEATURES + [TARGET_COL]).copy()

# Map tickers to ids (embedding input)
tickers = sorted(df["ticker"].unique().tolist())
ticker2id = {t:i for i,t in enumerate(tickers)}
df["ticker_id"] = df["ticker"].map(ticker2id).astype(int)

# Train/Test split by time 
train_df = df[df["Date"] < pd.to_datetime(TRAIN_END)].copy()
test_df  = df[df["Date"] >= pd.to_datetime(TRAIN_END)].copy()

# Fit scaler on TRAIN ONLY (prevents leakage)
scaler = StandardScaler()
scaler.fit(train_df[FEATURES].values)

def make_sequences(split_df: pd.DataFrame, seq_len: int):
    """
    Returns:
      X_seq: (N, seq_len, num_features) float32
      tid:   (N,) int64
      y:     (N,) float32
      meta:  DataFrame with Date,ticker aligned to each sample (optional)
    """
    X_list, tid_list, y_list = [], [], []
    meta_rows = []

    for tkr, g in split_df.groupby("ticker", sort=False):
        g = g.sort_values("Date").reset_index(drop=True)

        X = scaler.transform(g[FEATURES].values)        # scaled features
        y = g[TARGET_COL].values.astype(np.float32)
        tid = g["ticker_id"].iloc[0]

        # rolling windows
        for i in range(seq_len, len(g)):
            X_list.append(X[i-seq_len:i])
            tid_list.append(tid)
            y_list.append(y[i])
            meta_rows.append((g.loc[i, "Date"], tkr))

    X_seq = np.array(X_list, dtype=np.float32)
    tid   = np.array(tid_list, dtype=np.int64)
    y     = np.array(y_list, dtype=np.float32)
    meta  = pd.DataFrame(meta_rows, columns=["Date","ticker"])
    return X_seq, tid, y, meta

X_train, tid_train, y_train, meta_train = make_sequences(train_df, SEQ_LEN)
X_test,  tid_test,  y_test,  meta_test  = make_sequences(test_df, SEQ_LEN)

X_train.shape, X_test.shape

((43750, 60, 10), (14050, 60, 10))

In [24]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class SeqDataset(Dataset):
    def __init__(self, X, tid, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.tid = torch.tensor(tid, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self): return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.tid[idx], self.y[idx]

class LSTMWithTicker(nn.Module):
    def __init__(self, n_features, n_tickers, emb_dim=16, hidden=64, layers=2, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(n_tickers, emb_dim)

        self.lstm = nn.LSTM(
            input_size=n_features + emb_dim,
            hidden_size=hidden,
            num_layers=layers,
            batch_first=True,
            dropout=dropout if layers > 1 else 0.0
        )
        self.head = nn.Sequential(
            nn.Linear(hidden, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x, tid):
        # x: (B, T, F)
        B, T, F = x.shape
        e = self.emb(tid)                 # (B, emb_dim)
        e = e.unsqueeze(1).repeat(1, T, 1)# (B, T, emb_dim)
        xcat = torch.cat([x, e], dim=-1)  # (B, T, F+emb_dim)

        out, _ = self.lstm(xcat)          # (B, T, hidden)
        last = out[:, -1, :]              # last timestep
        yhat = self.head(last).squeeze(-1)
        return yhat

In [29]:
from sklearn.metrics import mean_absolute_error

device = "cuda" if torch.cuda.is_available() else "cpu"

train_loader = DataLoader(SeqDataset(X_train, tid_train, y_train), batch_size=512, shuffle=True)
test_loader  = DataLoader(SeqDataset(X_test,  tid_test,  y_test),  batch_size=1024, shuffle=False)

model = LSTMWithTicker(
    n_features=len(FEATURES),
    n_tickers=len(ticker2id),
    emb_dim=16,
    hidden=64,
    layers=2,
    dropout=0.2
).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = nn.SmoothL1Loss()  # robust for noisy returns

def predict_all(loader):
    model.eval()
    preds, ys = [], []
    with torch.no_grad():
        for xb, tidb, yb in loader:
            xb, tidb = xb.to(device), tidb.to(device)
            p = model(xb, tidb).cpu().numpy()
            preds.append(p)
            ys.append(yb.numpy())
    return np.concatenate(preds), np.concatenate(ys)

EPOCHS = 10
best_mae = float("inf")
best_epoch = None

for epoch in range(1, EPOCHS+1):
    model.train()
    total = 0.0
    for xb, tidb, yb in train_loader:
        xb, tidb, yb = xb.to(device), tidb.to(device), yb.to(device)
        opt.zero_grad()
        yhat = model(xb, tidb)
        loss = loss_fn(yhat, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        total += loss.item()

    preds, ys = predict_all(test_loader)
    mae = mean_absolute_error(ys, preds)
    da = (np.sign(preds) == np.sign(ys)).mean()

    print(f"Epoch {epoch:02d} | train_loss={total/len(train_loader):.5f} | MAE={mae:.5f} | DA={da:.3f}")

    if mae < best_mae:
        best_mae = mae
        best_epoch = epoch
        torch.save(model.state_dict(), "lstm_model.pt")
    

print("Best epoch:", best_epoch, "Best MAE:", best_mae)

Epoch 01 | train_loss=0.00027 | MAE=0.01190 | DA=0.526
Epoch 02 | train_loss=0.00018 | MAE=0.01217 | DA=0.528
Epoch 03 | train_loss=0.00018 | MAE=0.01175 | DA=0.527
Epoch 04 | train_loss=0.00017 | MAE=0.01189 | DA=0.528
Epoch 05 | train_loss=0.00017 | MAE=0.01172 | DA=0.490
Epoch 06 | train_loss=0.00017 | MAE=0.01176 | DA=0.520
Epoch 07 | train_loss=0.00016 | MAE=0.01175 | DA=0.487
Epoch 08 | train_loss=0.00016 | MAE=0.01178 | DA=0.529
Epoch 09 | train_loss=0.00016 | MAE=0.01167 | DA=0.492
Epoch 10 | train_loss=0.00016 | MAE=0.01165 | DA=0.530
Best epoch: 10 Best MAE: 0.011648581363260746


In [30]:
# Here we can see that LSTM model has slightly better MAE than RandomForest model, which means its daily return error is lower.
# In RF we had MAE=0.0119 whereas in LSTM we have MAE=0.0116 which is around 0.0003 ~ 2.5% improvement in error.

In [31]:
# Build test dataframe aligned with LSTM predictions
lstm_test_df = meta_test.copy()
lstm_test_df["target"] = y_test

lstm_test_df.head()

,Date,ticker,target
0,2022-03-30,AAPL,-0.017776
1,2022-03-31,AAPL,-0.001718
2,2022-04-01,AAPL,0.023694
3,2022-04-04,AAPL,-0.018942
4,2022-04-05,AAPL,-0.018451


In [32]:
thresholds = [0.0, 0.0005, 0.001, 0.002, 0.003]
rows = []

for th in thresholds:
    daily = run_backtest(
        test_df=lstm_test_df,
        preds=preds,
        cost_bps=5.0,
        mode="long_cash",
        signal_threshold=th
    )
    rows.append(summarize_backtest(daily, label=f"LSTM th={th}"))

lstm_sweep = pd.DataFrame(rows).sort_values("Strategy Sharpe", ascending=False)
lstm_sweep

,Label,Strategy Total Return,Buy&Hold Total Return,Strategy Sharpe,Buy&Hold Sharpe,Strategy Max Drawdown,Buy&Hold Max Drawdown,Avg Exposure,Avg Turnover,Days
1,LSTM th=0.0005,1.043805,1.014586,1.457565,1.387722,-0.166431,-0.169536,0.977173,0.021218,974
0,LSTM th=0.0,1.036079,1.014586,1.436958,1.387722,-0.160218,-0.169536,0.987064,0.012971,974
2,LSTM th=0.001,0.925918,1.014586,1.406240,1.387722,-0.161115,-0.169536,0.912457,0.059514,974
4,LSTM th=0.003,0.075687,1.014586,1.002116,1.387722,-0.021009,-0.169536,0.015024,0.014237,974
3,LSTM th=0.002,0.079138,1.014586,0.417266,1.387722,-0.070684,-0.169536,0.108179,0.067180,974


In [33]:
best_row = lstm_sweep.iloc[0]
best_th = float(best_row["Label"].split("=")[1])

print("Best LSTM threshold:", best_th)
best_row

Best LSTM threshold: 0.0005


Label                    LSTM th=0.0005
Strategy Total Return          1.043805
Buy&Hold Total Return          1.014586
Strategy Sharpe                1.457565
Buy&Hold Sharpe                1.387722
Strategy Max Drawdown         -0.166431
Buy&Hold Max Drawdown         -0.169536
Avg Exposure                   0.977173
Avg Turnover                   0.021218
Days                                974
Name: 1, dtype: object

In [35]:
# The LSTM model gave better results in backtesting when compared to RandomForest model.
# With the same best threshold of 0.0005, LSTM had better total returns, almost similar Sharpe ratio, slightly higher max drawdown but better exposure.

In [34]:
daily_lstm = run_backtest(
    test_df=lstm_test_df,
    preds=preds,
    cost_bps=5.0,
    mode="long_cash",
    signal_threshold=best_th
)

summarize_backtest(daily_lstm, label=f"LSTM th={best_th}")

plot_equity_curve(daily_lstm, f"LSTM Strategy (th={best_th})")
plot_equity_curve(daily_lc, "RF Strategy")
plot_equity_curve(daily_ls, "RF Long/Short Strategy")

In [36]:
# Over here we can see that Post April 2025 market, RF model started underperforming and couldn't capture the strong market trend, 
# whereas LSTM model was able to capture it better as it captures temporal patterns in data.

In [37]:
# Lets train the LSTM model as our primary model for our Streamlit app.
# We will increase the number of tickers per country to total 30, which will help the model generalize better across markets.

# Train LSTM model

In [38]:
FULL_STOCKS = {
    "US": ["AAPL", "MSFT", "AMZN", "NVDA", "JPM", "META", "XOM", "JNJ", "PG", "UNH", "KO"], # 11
    "Canada": ["RY.TO", "TD.TO", "ENB.TO", "CM.TO", "NA.TO", "CNQ.TO", "BCE.TO", "BNS.TO", "SU.TO", "CP.TO"], # 10
    "India": ["RELIANCE.NS", "TCS.NS", "INFY.NS","ADANIPOWER.NS", "HDFCBANK.NS", "ICICIBANK.NS", "LT.NS", "ITC.NS", "SBIN.NS"], # 9
}

In [39]:
def prepare_stock_df(ticker, start="2010-01-01", end="2025-12-31"):
    df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)
    if df is None or df.empty:
        return None

    # If yfinance ever returns MultiIndex columns, flatten them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df = df.reset_index()  # adds Date column
    close = df["Close"].squeeze() # makes sure to make a series instead of dataframe

    # Returns
    df["ret"] = close.pct_change()
    df["ret_1"] = df["ret"].shift(1)
    df["ret_5"] = df["ret"].shift(5)
    df["ret_20"] = df["ret"].shift(20)

    # Volatility
    df["vol20"] = df["ret"].rolling(20).std()

    # RSI
    delta = close.diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = (-delta.clip(upper=0)).rolling(14).mean()
    rs = gain / loss
    df["RSI14"] = 100 - (100 / (1 + rs))

    # MACD
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    df["MACD"] = ema12 - ema26

    # Bollinger width (normalized)
    m20 = close.rolling(20).mean()
    s20 = close.rolling(20).std()
    df["bb_width"] = (4 * s20) / m20   # (BB_up - BB_low) / mid

    # ATR %
    high, low = df["High"].squeeze(), df["Low"].squeeze()
    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

    df["ATR14"] = tr.rolling(14).mean()
    df["atr_pct"] = df["ATR14"] / close

    # Target: next-day return
    df["target"] = df["ret"].shift(-1)

    df["ticker"] = ticker

    keep = ["Date", "ticker", "Close", "High", "Low", "Open", "Volume",
            "ret", "ret_1", "ret_5", "ret_20", "vol20", "RSI14", "MACD",
            "bb_width", "ATR14", "atr_pct", "target"]

    df = df[keep].dropna()

    return df

In [40]:
all_data = []

for market, tickers in FULL_STOCKS.items():
    for t in tickers:
        print(f"Fetching {t}")
        df_t = prepare_stock_df(t)
        if df_t is not None and not df_t.empty:
            all_data.append(df_t)

final_df = pd.concat(all_data, ignore_index=True)

Fetching AAPL
Fetching MSFT
Fetching AMZN
Fetching NVDA
Fetching JPM
Fetching META
Fetching XOM
Fetching JNJ
Fetching PG
Fetching UNH
Fetching KO
Fetching RY.TO
Fetching TD.TO
Fetching ENB.TO
Fetching CM.TO
Fetching NA.TO
Fetching CNQ.TO
Fetching BCE.TO
Fetching BNS.TO
Fetching SU.TO
Fetching CP.TO
Fetching RELIANCE.NS
Fetching TCS.NS
Fetching INFY.NS
Fetching ADANIPOWER.NS
Fetching HDFCBANK.NS
Fetching ICICIBANK.NS
Fetching LT.NS
Fetching ITC.NS
Fetching SBIN.NS


In [41]:
final_df[final_df["ticker"]=="LT.NS"]

Price,Date,ticker,Close,High,Low,Open,Volume,ret,ret_1,ret_5,ret_20,vol20,RSI14,MACD,bb_width,ATR14,atr_pct,target
106890,2010-02-03,LT.NS,517.681824,520.127162,496.209172,500.260943,6031122,0.038492,-0.014712,-0.021738,0.001774,0.020287,20.271972,-25.102193,0.285745,18.454837,0.035649,-0.003689
106891,2010-02-04,LT.NS,515.772034,520.091528,508.917878,514.058483,3639273,-0.003689,0.038492,-0.018210,-0.011066,0.020287,20.242303,-24.520479,0.288525,18.559386,0.035984,-0.014397
106892,2010-02-05,LT.NS,508.346649,514.986567,496.173442,505.133784,4964971,-0.014397,-0.003689,-0.005344,-0.004804,0.020341,20.050174,-24.377623,0.292256,19.218540,0.037806,0.014361
106893,2010-02-08,LT.NS,515.646973,521.912049,504.598288,514.058385,3517267,0.014361,-0.014397,-0.004635,0.006326,0.020708,23.460416,-23.405529,0.288015,19.613775,0.038037,0.012046
106894,2010-02-09,LT.NS,521.858521,522.929459,511.648742,517.271256,3634060,0.012046,0.014361,-0.014712,-0.000506,0.021110,27.494755,-21.881679,0.278485,19.622701,0.037602,-0.017546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110813,2025-12-23,LT.NS,4058.800049,4095.000000,4053.300049,4089.000000,1867147,-0.003340,-0.000270,-0.006964,-0.004136,0.008494,60.832335,25.430335,0.035878,54.785679,0.013498,-0.001281
110814,2025-12-24,LT.NS,4053.600098,4080.699951,4048.100098,4065.000000,1021436,-0.001281,-0.003340,-0.000345,0.016338,0.007673,60.683773,23.830916,0.035690,53.721383,0.013253,0.000000
110815,2025-12-25,LT.NS,4053.600098,4053.600098,4053.600098,4053.600098,0,0.000000,-0.001281,-0.007705,0.004751,0.007588,52.820542,22.306232,0.034632,48.514247,0.011968,-0.001554
110816,2025-12-26,LT.NS,4047.300049,4061.500000,4030.199951,4052.000000,594330,-0.001554,0.000000,0.010518,-0.002867,0.007571,60.639222,20.354909,0.034018,45.678537,0.011286,-0.002125


In [58]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# CONFIG
SEQ_LEN = 60
TRAIN_END = "2022-01-01"   # train < this date, test >= this date

FEATURES = [
    "ret", "ret_1", "ret_5", "ret_20",
    "vol20", "RSI14", "MACD", "bb_width", "ATR14", "atr_pct"
]
TARGET_COL = "target"      # next-day return


# Load / clean / sort
df = final_df.copy()

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["ticker", "Date"]).reset_index(drop=True)

# Drop rows with missing features/target
df = df.dropna(subset=FEATURES + [TARGET_COL]).copy()


# Train/Test split by time
train_df = df[df["Date"] < pd.to_datetime(TRAIN_END)].copy()
test_df  = df[df["Date"] >= pd.to_datetime(TRAIN_END)].copy()


# Fit scaler on TRAIN 
scaler = StandardScaler()
scaler.fit(train_df[FEATURES].values)


# Sequence builder (NO ticker embedding)
# We remove ticker embedding to help model support any new ticker at inference time.
def make_sequences(split_df: pd.DataFrame, seq_len: int):
    """
    Returns:
      X_seq: (N, seq_len, num_features) float32
      y:     (N,) float32
      meta:  DataFrame with Date,ticker aligned to each sample (for backtesting)
    """
    X_list, y_list = [], []
    meta_rows = []

    # Build windows per ticker to avoid mixing time series
    for tkr, g in split_df.groupby("ticker", sort=False):
        g = g.sort_values("Date").reset_index(drop=True)

        # Scale using TRAIN-fitted scaler
        X = scaler.transform(g[FEATURES].values).astype(np.float32)
        y = g[TARGET_COL].values.astype(np.float32)

        # Rolling windows: predict y[i] using X[i-seq_len : i]
        for i in range(seq_len, len(g)):
            X_list.append(X[i - seq_len : i])
            y_list.append(y[i])
            meta_rows.append((g.loc[i, "Date"], tkr))

    X_seq = np.array(X_list, dtype=np.float32)
    y_out = np.array(y_list, dtype=np.float32)
    meta  = pd.DataFrame(meta_rows, columns=["Date", "ticker"])
    return X_seq, y_out, meta

# Build sequences
X_train, y_train, meta_train = make_sequences(train_df, SEQ_LEN)
X_test,  y_test,  meta_test  = make_sequences(test_df, SEQ_LEN)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test :", X_test.shape,  "y_test :", y_test.shape)


X_train: (86961, 60, 10) y_train: (86961,)
X_test : (28113, 60, 10) y_test : (28113,)


In [59]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

# Dataset (NO ticker embedding)
class SeqDataset(Dataset):
    """
    Returns (X_seq, y) where:
      X_seq: (seq_len, n_features) float32
      y:     scalar float32 (next-day return)
    """
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# LSTM Model (NO ticker embedding)
class LSTMModel(nn.Module):
    def __init__(self, n_features, hidden=64, layers=2, dropout=0.2):
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=hidden,
            num_layers=layers,
            batch_first=True,
            dropout=dropout if layers > 1 else 0.0
        )

        self.head = nn.Sequential(
            nn.Linear(hidden, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # x: (B, T, F)
        out, _ = self.lstm(x)     # (B, T, hidden)
        last = out[:, -1, :]      # (B, hidden)
        yhat = self.head(last).squeeze(-1)  # (B,)
        return yhat

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import mean_absolute_error

# Device setup (GPU if available, else CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# DataLoaders
# We shuffle training data (better generalization)
# We do NOT shuffle test data (preserve time order)
train_loader = DataLoader(
    SeqDataset(X_train, y_train),
    batch_size=512,
    shuffle=True
)

test_loader = DataLoader(
    SeqDataset(X_test, y_test),
    batch_size=1024,
    shuffle=False
)

# Initialize LSTM model
# No ticker embedding -> model can generalize to ANY ticker
model = LSTMModel(
    n_features=len(FEATURES),
    hidden=64,
    layers=2,
    dropout=0.2
).to(device)


# Optimizer + loss
# AdamW: stable for noisy financial data
# SmoothL1Loss (Huber): robust to outliers in returns, unlike MSE that over-penalizes large errors
opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = nn.SmoothL1Loss()


# Helper function: get predictions on test set
# Used every epoch to monitor generalization
def predict_all(loader):
    model.eval()
    preds, ys = [], []

    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)

            p = model(xb)                # forward pass
            preds.append(p.cpu().numpy())
            ys.append(yb.cpu().numpy())

    return np.concatenate(preds), np.concatenate(ys)

# Training loop with early-best saving
# We select the model with the LOWEST validation MAE
EPOCHS = 10
best_mae = float("inf")
best_epoch = None

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    # Training step
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)

        opt.zero_grad()

        yhat = model(xb)               # forward
        loss = loss_fn(yhat, yb)       # loss
        loss.backward()                # backprop

        # Gradient clipping prevents exploding gradients in RNNs, as it stabilizes training and avoids NaNs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        opt.step()
        total_loss += loss.item()

    # Validation / test evaluation 
    preds, ys = predict_all(test_loader)

    mae = mean_absolute_error(ys, preds)
    da = (np.sign(preds) == np.sign(ys)).mean()   # directional accuracy

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={total_loss/len(train_loader):.5f} | "
        f"MAE={mae:.5f} | "
        f"DA={da:.3f}"
    )

    # Save best model
    if mae < best_mae:
        best_mae = mae
        best_epoch = epoch
        torch.save(model.state_dict(), "lstm_model.pt")

print("Best epoch:", best_epoch, "Best MAE:", best_mae)


Epoch 01 | train_loss=0.00019 | MAE=0.01117 | DA=0.511
Epoch 02 | train_loss=0.00015 | MAE=0.01115 | DA=0.476
Epoch 03 | train_loss=0.00015 | MAE=0.01107 | DA=0.524
Epoch 04 | train_loss=0.00015 | MAE=0.01107 | DA=0.525
Epoch 05 | train_loss=0.00015 | MAE=0.01109 | DA=0.523
Epoch 06 | train_loss=0.00015 | MAE=0.01108 | DA=0.524
Epoch 07 | train_loss=0.00015 | MAE=0.01108 | DA=0.523
Epoch 08 | train_loss=0.00015 | MAE=0.01108 | DA=0.522
Epoch 09 | train_loss=0.00015 | MAE=0.01111 | DA=0.476
Epoch 10 | train_loss=0.00015 | MAE=0.01108 | DA=0.524
Best epoch: 4 Best MAE: 0.011066492646932602


In [61]:
# This was the result using 15 tickers, Best MAE: 0.011648581363260746
# With 30 tickers, we got Best MAE: 0.011066492646932602 which is around 5% improvement in error.

In [ ]:
import numpy as np
import pandas as pd

def max_drawdown(equity_curve: pd.Series) -> float:
    # Max drawdown in % terms (negative number).

    peak = equity_curve.cummax()
    dd = equity_curve / peak - 1.0
    return float(dd.min())

def sharpe_ratio(daily_returns: pd.Series, rf_daily: float = 0.0) -> float:
    # Annualized Sharpe ratio based on daily returns.
    
    r = daily_returns.dropna() - rf_daily
    if r.std() == 0 or len(r) < 2:
        return float("nan")
    return float((r.mean() / r.std()) * np.sqrt(252))

def run_backtest_rebalance(
    test_df: pd.DataFrame,
    preds: np.ndarray,
    cost_bps: float = 5.0,
    mode: str = "long_cash",
    signal_threshold: float = 0.0,
    rebalance_every: int = 5,   # 5 = weekly, 21 = monthly-ish
) -> pd.DataFrame:
    """
    Backtest with discrete rebalancing (trade only every N trading days per ticker).

    Parameters
    ----------
    test_df : DataFrame
        Must contain ['Date','ticker','target'] where target = next-day return.
    preds : np.ndarray
        Model predicted next-day returns aligned with test_df rows.
    cost_bps : float
        Transaction cost per trade in basis points (bps). Applied when position changes.
    mode : str
        'long_cash' -> position in {0, +1}
        'long_short'-> position in {-1, +1}
    signal_threshold : float
        Only take trades when pred > threshold (reduces churn).
    rebalance_every : int
        Only update positions every N rows per ticker (e.g., 5 for weekly).
        Between rebalance days, hold the previous position.

    Returns
    -------
    DataFrame with daily portfolio returns and equity curves.
    """

    bt = test_df[["Date", "ticker", "target"]].copy()
    bt["pred"] = preds

    bt = bt.sort_values(["ticker", "Date"]).reset_index(drop=True)

    # Build raw "desired" signal each day 
    if mode == "long_cash":
        bt["desired_pos"] = (bt["pred"] > signal_threshold).astype(int)  # 1 or 0
    elif mode == "long_short":
        bt["desired_pos"] = np.where(bt["pred"] > signal_threshold, 1, -1)  # 1 or -1
    else:
        raise ValueError("mode must be 'long_cash' or 'long_short'")

    # Only allow position changes every N days per ticker 
    # rebalance_idx: 0,1,2,... within each ticker
    bt["rebalance_idx"] = bt.groupby("ticker").cumcount()

    # We update pos only on rows where rebalance_idx % N == 0
    reb_mask = (bt["rebalance_idx"] % rebalance_every) == 0

    # Start with NaN, set desired_pos on rebalance days, then forward-fill (hold position)
    bt["pos"] = np.nan
    bt.loc[reb_mask, "pos"] = bt.loc[reb_mask, "desired_pos"]
    bt["pos"] = bt.groupby("ticker")["pos"].ffill().fillna(0)  # before first rebalance -> 0 (cash)

    # Costs: pay when position changes (only happens on rebalance days) 
    bt["pos_prev"] = bt.groupby("ticker")["pos"].shift(1).fillna(0)
    bt["turnover"] = (bt["pos"] - bt["pos_prev"]).abs()

    cost = (cost_bps / 10000.0)
    bt["cost"] = bt["turnover"] * cost

    # Strategy returns 
    bt["strat_ret"] = bt["pos"] * bt["target"] - bt["cost"]
    bt["bh_ret"] = bt["target"]

    # Aggregate to portfolio daily (equal-weight across tickers present) 
    daily = (
        bt.groupby("Date", as_index=False)[["strat_ret", "bh_ret"]]
        .mean()
        .sort_values("Date")
    )
    daily["strat_equity"] = (1 + daily["strat_ret"]).cumprod()
    daily["bh_equity"] = (1 + daily["bh_ret"]).cumprod()

    
    daily["Avg Exposure"] = bt.groupby("Date")["pos"].mean().reindex(daily["Date"]).values
    daily["Avg Turnover"] = bt.groupby("Date")["turnover"].mean().reindex(daily["Date"]).values

    return daily

def summarize_backtest(daily: pd.DataFrame, label: str = "") -> dict:
    """Compute key metrics from daily backtest results."""
    strat_total = float(daily["strat_equity"].iloc[-1] - 1)
    bh_total = float(daily["bh_equity"].iloc[-1] - 1)

    strat_sharpe = sharpe_ratio(daily["strat_ret"])
    bh_sharpe = sharpe_ratio(daily["bh_ret"])

    strat_mdd = max_drawdown(daily["strat_equity"])
    bh_mdd = max_drawdown(daily["bh_equity"])

    return {
        "Label": label,
        "Strategy Total Return": strat_total,
        "Buy&Hold Total Return": bh_total,
        "Strategy Sharpe": strat_sharpe,
        "Buy&Hold Sharpe": bh_sharpe,
        "Strategy Max Drawdown": strat_mdd,
        "Buy&Hold Max Drawdown": bh_mdd,
        "Avg Exposure": float(np.nanmean(daily["Avg Exposure"])),
        "Avg Turnover": float(np.nanmean(daily["Avg Turnover"])),
        "Days": int(len(daily)),
    }

In [63]:
# Build test dataframe aligned with LSTM predictions
lstm_test_df = meta_test.copy()
lstm_test_df["target"] = y_test

lstm_test_df.head()

,Date,ticker,target
0,2022-03-30,AAPL,-0.017776
1,2022-03-31,AAPL,-0.001718
2,2022-04-01,AAPL,0.023694
3,2022-04-04,AAPL,-0.018942
4,2022-04-05,AAPL,-0.018451


In [67]:
rebalance_days = [1, 2, 5, 10, 21]
thresholds = [0.0, 0.0002, 0.0005, 0.0008, 0.001, 0.0015, 0.002]

rows = []
for rb in rebalance_days:
    for th in thresholds:
        d = run_backtest_rebalance(
            test_df=lstm_test_df,
            preds=preds,
            cost_bps=5.0,
            mode="long_cash",
            signal_threshold=th,
            rebalance_every=rb
        )
        rows.append(summarize_backtest(d, label=f"LSTM rb={rb}, th={th}"))

pd.DataFrame(rows).sort_values("Strategy Sharpe", ascending=False).head(10)

,Label,Strategy Total Return,Buy&Hold Total Return,Strategy Sharpe,Buy&Hold Sharpe,Strategy Max Drawdown,Buy&Hold Max Drawdown,Avg Exposure,Avg Turnover,Days
0,"LSTM rb=1, th=0.0",0.791313,0.773503,1.421571,1.385249,-0.132232,-0.129089,0.992405,0.012297,974
2,"LSTM rb=1, th=0.0005",0.774724,0.773503,1.407523,1.385249,-0.130679,-0.129089,0.985253,0.019146,974
7,"LSTM rb=2, th=0.0",0.772903,0.773503,1.402064,1.385249,-0.136759,-0.129089,0.992560,0.007212,974
1,"LSTM rb=1, th=0.0002",0.776043,0.773503,1.401721,1.385249,-0.132723,-0.129089,0.991550,0.013067,974
8,"LSTM rb=2, th=0.0002",0.755722,0.773503,1.378561,1.385249,-0.137256,-0.129089,0.991585,0.007760,974
9,"LSTM rb=2, th=0.0005",0.749731,0.773503,1.378118,1.385249,-0.134689,-0.129089,0.985169,0.011326,974
28,"LSTM rb=21, th=0.0",0.750265,0.773503,1.369529,1.385249,-0.126704,-0.129089,0.992569,0.002019,974
14,"LSTM rb=5, th=0.0",0.741484,0.773503,1.354678,1.385249,-0.129446,-0.129089,0.993199,0.003624,974
15,"LSTM rb=5, th=0.0002",0.740350,0.773503,1.353332,1.385249,-0.128734,-0.129089,0.991813,0.004103,974
29,"LSTM rb=21, th=0.0002",0.736817,0.773503,1.351150,1.385249,-0.126704,-0.129089,0.991112,0.002156,974


In [66]:
daily_lstm = run_backtest_rebalance(
    test_df=lstm_test_df,
    preds=preds,
    cost_bps=0.0,
    mode="long_cash",
    signal_threshold=0.00,
    rebalance_every=1
)

summarize_backtest(daily_lstm, label=f"LSTM th={best_th}")

plot_equity_curve(daily_lstm, f"LSTM Strategy (th={best_th})")
plot_equity_curve(daily_lc, "RF Strategy")
plot_equity_curve(daily_ls, "RF Long/Short Strategy")

In [68]:
# Finally our model was able to beat the Buy & Hold Strategy and our LSTM model is finally complete.

# Building Artifacts for Streamlit

In [ ]:
import os, json
import joblib
from pathlib import Path

ART_DIR = Path("D:\\Stock Trend Prediction\\artifacts")
ART_DIR.mkdir(exist_ok=True)

# Save model weights 
# If lstm_model.pt is already in the same folder as the notebook, this moves/copies it.
src = Path("D:\\Stock Trend Prediction\\artifacts\\lstm_model.pt")
dst = ART_DIR / "lstm_model.pt"
if src.exists():
    # copy (safe).
    dst.write_bytes(src.read_bytes())
else:
    raise FileNotFoundError("lstm_model.pt not found in current working directory.")

# Save the scaler used for LSTM feature scaling
joblib.dump(scaler, ART_DIR / "lstm_scaler.pkl")

# Save the exact feature list
with open(ART_DIR / "lstm_features.json", "w") as f:
    json.dump(FEATURES, f, indent=2)

# Save config used in Streamlit for inference + strategy display
lstm_config = {
    "seq_len": SEQ_LEN,
    "train_end": TRAIN_END,
    "signal_threshold": 0.0,     
    "rebalance_every": 1,        
    "cost_bps": 5.0,             
    "model_type": "LSTMModel_no_embedding"
}
with open(ART_DIR / "lstm_config.json", "w") as f:
    json.dump(lstm_config, f, indent=2)

# saves list of tickers used in training (for transparency/debugging)
trained_tickers = sorted(df["ticker"].unique().tolist())  # df is your cleaned dataframe used for sequences
with open(ART_DIR / "trained_tickers.json", "w") as f:
    json.dump(trained_tickers, f, indent=2)

print("Artifacts saved to:", ART_DIR.resolve())
print("Files:", [p.name for p in ART_DIR.iterdir()])


Artifacts saved to: D:\Stock Trend Prediction\artifacts
Files: ['lstm_config.json', 'lstm_features.json', 'lstm_model.pt', 'lstm_scaler.pkl', 'trained_tickers.json']
